In [1]:
import argparse
import random
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from dataset import trafficDataLoader
from model import GRNN
from utils import Log

from types import SimpleNamespace

In [2]:
#TODO: reorganize and clean code up

print('Loading data . . .')
dataLoader = trafficDataLoader(1, 10)
data = dataLoader.data
print('Done loading data . . .')

print(data.shape)

# scale data using MinMaxScaler
print('Scaling data . . .')
scaler = MinMaxScaler()
scaler.fit(data)
scaler_data = scaler.transform(data)
print('Done scaling data . . .')

data = np.transpose(scaler_data)  # [T, n]
data = data[np.newaxis, :, :, np.newaxis]
data = torch.from_numpy(data)                                           # [b, T, n, d]

Loading data . . .
Done loading data . . .
(111, 6336)
Scaling data . . .
Done scaling data . . .


In [3]:
# reference for opt: hardcoding to test and then make more dynamic later
# batchSize = 1
# nNode = len selSegs
# dimFeature = 1
# dimHidden = 25 (see big D in pt file)
# interval = big T
# useCuda

opt = {
    'batchSize': 1,
    'nNode': 111, # update this later to read the number of rows in the selsegs file
    'dimFeature': 1,
    'dimHidden': 25, # read big D from the pt file
    'truncate': 48, # read big T from pt file
    'cuda': True,
    'lr': .05, # read lr from pt file
    'interval': data.size(1)
}

# convert the dictionary so we can use . syntax to reference its keys
opt = SimpleNamespace(**opt)

In [7]:
# load A and set 1 on the diagonal and zeros elsewhere of each node
A = dataLoader.A
# A = opt.alpha * A + np.eye(opt.nNode) # original
A = np.eye(opt.nNode) # been training with this, no problem yet

A = torch.from_numpy(A[np.newaxis, :, :])                               # [b, n, n]
hState = torch.randn(opt.batchSize, opt.dimHidden, opt.nNode).double()  # [b, D, n]

net = GRNN(opt)
optimizer = optim.Adam(net.parameters(), lr=opt.lr)

print('Loading model . . .')
checkpoint = torch.load('./result/grnn29055-30int-1tid-0.0001a-48T-25D-4i-0.05lr-5487ms-1b-18sn.pt')
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print('Done loading model . . .')

# choose to run model in evaluation or training mode
# set model to evaluation mode
net.eval()

# continue training the model
# net.train()

net.double()
criterion = nn.MSELoss()

if opt.cuda:
    net.cuda()
    criterion.cuda()
    data = data.cuda()
    A = A.cuda()
    hState = hState.cuda()

# use cuDNN along with cuda to train faster
torch.backends.cudnn.benchmark = True

Loading model . . .
Done loading model . . .


In [11]:
# start from the next index after 70% of the dataset is trained
starting_index = int(opt.interval * .7) - opt.truncate
ending_index = int(opt.interval) - opt.truncate

predictions = np.zeros((opt.nNode, ending_index - starting_index))
index = 0

# starting index for checking accuracies
reference_index = starting_index + opt.truncate

print('Beginning predictions . . .')
# for t in range(opt.interval - opt.truncate):
for t in range(starting_index, ending_index, 1):
    x = data[:, t:(t + opt.truncate), :, :] # batch, interval, node, feature
    O, _ = net(x, hState, A) # data, hState, A

    # rescale O before printing preditcions
    transform_O_temp = np.zeros((opt.nNode, opt.interval))
    transform_O_temp[:, :opt.truncate] = np.reshape(O[0, :, :, 0].cpu().data.numpy().flatten(), (opt.nNode, opt.truncate))
    O = scaler.inverse_transform(transform_O_temp)[:, :opt.truncate]
    
    # store the predictions
    predictions[:, index] = O[:, -1]
    index += 1


Beginning predictions . . .


In [12]:
predictions.shape

(111, 1901)

In [29]:
accuracies = []
for j in range(0, index, 1):
    accuracy_score = 0
    test_comparison_data = dataLoader.data[:, reference_index + j]
    
    for i in range(opt.nNode):
        # give an accuracy of 0 if the test comparison data is 0
        if test_comparison_data[i] == 0:
            accuracy_score += 0
        else:
            accuracy_score += predictions[i, j] / test_comparison_data[i]

    accuracy_score /= opt.nNode
    accuracies.append(round(accuracy_score * 100.0, 2))


print(f'Accuracy scores: {accuracies}')

Accuracy scores: [86.95, 86.5, 87.02, 86.4, 86.31, 129.34, 129.26, 127.69, 128.23, 126.3, 124.79, 126.21, 125.61, 126.01, 125.01, 125.12, 125.35, 126.03, 126.18, 124.61, 127.0, 125.88, 127.29, 77.62, 77.59, 78.34, 77.1, 78.29, 78.5, 77.73, 78.17, 78.33, 77.13, 78.29, 78.64, 92.19, 92.55, 91.73, 92.68, 93.79, 91.86, 93.31, 93.78, 93.18, 94.01, 95.44, 94.12, 93.54, 94.92, 98.79, 95.82, 94.91, 98.11, 97.42, 98.71, 98.65, 99.94, 99.67, 98.33, 80.7, 81.47, 81.26, 82.37, 81.45, 81.65, 79.77, 81.95, 81.16, 81.13, 81.07, 82.89, 84.44, 85.01, 84.47, 83.96, 84.19, 83.82, 85.16, 84.39, 84.97, 84.6, 84.35, 83.97, 84.09, 84.09, 84.56, 85.16, 85.12, 84.61, 109.84, 110.15, 109.72, 109.83, 110.29, 109.25, 108.15, 107.2, 107.78, 107.83, 107.34, 106.54, 107.47, 107.35, 106.49, 107.07, 107.43, 105.45, 86.69, 86.45, 86.35, 87.08, 85.59, 85.06, 85.55, 85.12, 84.7, 84.25, 83.06, 84.15, 84.76, 84.96, 84.56, 85.56, 84.24, 83.96, 89.74, 90.24, 89.74, 88.99, 89.62, 89.07, 89.44, 89.19, 89.94, 90.09, 89.17, 90.4

In [32]:
from functools import reduce
average_accuracy = reduce(lambda x, y: x + y, accuracies) / len(accuracies)

In [33]:
average_accuracy

96.12988953182551